In [1]:
import pandas as pd
import numpy as np
import Utilidades as ut

In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_csv("Archivos/Train_TP2_Modified.csv")
#entrenamiento = pd.read_csv("Archivos/Train_TP2_Datos_2020-2C.csv")
test = pd.read_csv("Archivos/Test_TP2_Datos_2020-2C.csv")

In [3]:
entrenamiento.columns

Index(['Region', 'ID', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Billing_Country',
       'Account_Name', 'Opportunity_Name', 'Opportunity_ID',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Quote_Expiry_Date', 'Last_Modified_Date',
       'Last_Modified_By', 'Product_Family', 'Product_Name', 'ASP_Currency',
       'ASP', 'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'TRF', 'Total_Amount_Currency',
       'Total_Amount', 'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount',
       'Stage'],
      dtype='object')

In [4]:
ut.diagnostico_df(entrenamiento)

Suma: 6973, Columna: Sales_Contract_No


1. Completar los Territory incompletos que tienen como region a Japan, con Japan

In [5]:
#FILTRADO DE OPORTUNIDADES QUE NO SON CLOSE WON NI CLOSE LOST

entrenamiento = entrenamiento[( entrenamiento['Stage'] == 'Closed Won') | ( entrenamiento['Stage'] == 'Closed Lost')]

In [6]:
#FORMATO FECHAS

#Respalda fecha, usada para separa entrenamiento y test

columnas_fecha = ['Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']

def formato_fechas(x):
    for columna in columnas_fecha:
        x[columna] = pd.to_datetime(x[columna])
        
formato_fechas(entrenamiento)
formato_fechas(test)

In [7]:
#ORDENO POR FECHA AMBOS DATAFRAMES

entrenamiento = entrenamiento.sort_values(by=['Opportunity_Created_Date'])
test = test.sort_values(by=['Opportunity_Created_Date'])

In [8]:
#RECUPERACION Y CONVERSION DE COLUMNAS

def recuperacion_y_conversion(df):
    #Arreglo el nombre de la columna 'Source'
    df = df.rename(columns={'Source ' : 'Source'})

    # Convierto Total_Amount y Total_Taxable_Amount a dolares, filtro columnas innecesarias (codigo extraido de TP1)

    df['Convertion_Factor'] = df['ASP_(converted)'] / df['ASP']

    #Intentamos salvar algunas entradas reemplazando con la media de la oportunidad

    df['Convertion_Factor_AVG'] = df.groupby('Opportunity_ID')['Convertion_Factor'].transform('mean')
    df['Convertion_Factor_is_null'] = df['Convertion_Factor'].isnull()
    df['Convertion_Factor'] = np.where(df['Convertion_Factor_is_null'] == 1, df['Convertion_Factor_AVG'], df['Convertion_Factor'])
    df['Convertion_Factor_AVG'] = df.groupby('ASP_Currency')['Convertion_Factor'].transform('mean')
    df['Convertion_Factor_is_null'] = df['Convertion_Factor'].isnull()
    df['Convertion_Factor'] = np.where(df['Convertion_Factor_is_null'] == 1, df['Convertion_Factor_AVG'], df['Convertion_Factor'])

    #Eliminamos las entradas que no pudimos salvar.
    df = df.loc[df.Convertion_Factor.isnull() == 0]

    #Eliminamos infinitos
    df = df.loc[df.Convertion_Factor != np.inf]

    # Eliminamos todos los 'Opportunity_ID' que tienen algun registro con Total_Amount == None 
    df['Total_Amount_is_null'] = df.Total_Amount.isnull().astype(int)
    df['Total_Amount_is_null'] = df.groupby('Opportunity_ID')['Total_Amount_is_null'].transform('sum')
    df = df.loc[df.Total_Amount_is_null == 0]

    #Convertimos los valores a dolares
    df['Total_Amount(USD)'] = df['Total_Amount'] * df['Convertion_Factor']
    df['Total_Taxable_Amount(USD)'] = df['Total_Taxable_Amount'] * df['Convertion_Factor']

    #Rescatamos ASP
    df['ASP_(converted)_avg'] = df.groupby('Region')['ASP_(converted)'].transform('mean')
    df['ASP_(converted)_is_null'] = df['ASP_(converted)'].isnull()
    df['ASP_(converted)'] = np.where(df['ASP_(converted)_is_null'] == 1, df['ASP_(converted)_avg'], df['ASP_(converted)'])

    #Eliminamos columnas utilizadas

    df = df.drop(columns={'Total_Amount_is_null', 
                          'Convertion_Factor_AVG',
                          'Convertion_Factor',
                          'Convertion_Factor_is_null',
                          'ASP_(converted)_avg',
                          'ASP_(converted)_is_null'})
    
    # Intentamos recuperar los datos faltantes

    #ACA LE PUEDO HABER PIFIADO
        
    # Billing Country y #Account_Type
    idx = df[df.Billing_Country == 'None'].index

    # Se completan los campos faltantes en base a la moda de la region.
    # Se intentaron completar a partir del nombre de cuenta, nombre de oportunidad e id de oportunidad, pero solo aparecia esta entrada.
    df.loc[idx, 'Billing_Country'] = df[df.Region == 'APAC']['Billing_Country'].mode()[0]
    df.loc[idx, 'Account_Type'] = df[df.Region == 'APAC']['Account_Type'].mode()[0]

    # Planned_Delivery_End_Date

    # Ya que son oportunidades de un unico producto, podemos predecir una fecha de fin de entrega a partir de la media de la region y familia de producto

    df['Planned_Delivery_Duration'] = (df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']) / np.timedelta64(1, 'D')
    df['Planned_Delivery_Duration_AVG'] = np.round(df.groupby(['Region', 'Product_Family'])['Planned_Delivery_Duration'].transform('mean'))
    df['Planned_Delivery_Duration_AVG'] = pd.to_timedelta(df['Planned_Delivery_Duration_AVG'], 'D')
    df['Predicted_Planned_Delivery_End_Date'] = df['Planned_Delivery_Start_Date'] + df['Planned_Delivery_Duration_AVG']

    df['Planned_Delivery_End_Date_is_null'] = df['Planned_Delivery_End_Date'].isnull()
    df['Planned_Delivery_End_Date'] = np.where(df['Planned_Delivery_End_Date_is_null'] == 1, df['Predicted_Planned_Delivery_End_Date'], df['Planned_Delivery_End_Date'])

    # Completamos las que quedaron con la media de la region, sin tener en cuenta el tipo de producto

    df['Planned_Delivery_Duration_AVG'] = round(df.groupby('Region')['Planned_Delivery_Duration'].transform('mean'))
    df['Planned_Delivery_Duration_AVG'] = pd.to_timedelta(df['Planned_Delivery_Duration_AVG'], 'D')
    df['Predicted_Planned_Delivery_End_Date'] = df['Planned_Delivery_Start_Date'] + df['Planned_Delivery_Duration_AVG']

    df['Planned_Delivery_End_Date_is_null'] = df['Planned_Delivery_End_Date'].isnull()
    df['Planned_Delivery_End_Date'] = np.where(df['Planned_Delivery_End_Date_is_null'] == 1, df['Predicted_Planned_Delivery_End_Date'], df['Planned_Delivery_End_Date'])

    df = df.drop(columns = ['Planned_Delivery_Duration',
                                      'Planned_Delivery_Duration_AVG',
                                      'Predicted_Planned_Delivery_End_Date', 
                                      'Planned_Delivery_End_Date_is_null'])
    
    return df

entrenamiento = recuperacion_y_conversion(entrenamiento)
test = recuperacion_y_conversion(test)

In [9]:
#LIMPIEZA COLUMNAS IRRECUPERABLES

drop_columnas = ['ASP_(converted)_Currency',
                 'Quote_Type',
                 'Brand',
                 'Product_Type',
                 'Size',
                 'Product_Category_B',
                 'Price',
                 'Currency',
                 'Last_Activity',
                 'Actual_Delivery_Date',
                 'Prod_Category_A',
                 'ID',
                 'Source',
                 'Quote_Expiry_Date',
                 'Sales_Contract_No']

for col in drop_columnas:
    if col in entrenamiento.columns:
        entrenamiento = entrenamiento.drop(columns=col)
        
for col in drop_columnas:
    if col in test.columns:
        test = test.drop(columns=col)
#entrenamiento = entrenamiento.drop(columns=drop_columnas)
#test = test.drop(columns=drop_columnas)

In [10]:
# COLUMNAS OBJECT A CATEGORY

def object_a_category(df):
    columnas_object = list(df.select_dtypes(include=['object']).columns)
    for columna in columnas_object:
        df[columna] = df[columna].astype('category')
    return df

entrenamiento = object_a_category(entrenamiento)
test = object_a_category(test)

In [11]:
# GUARDADO ARCHIVOS TEMPORALES
entrenamiento.to_pickle("Archivos/Entrenamiento_limpio.pkl")
test.to_pickle("Archivos/Test_limpio.pkl")

In [12]:
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16655 entries, 16104 to 16102
Data columns (total 39 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Region                              16655 non-null  category      
 1   Territory                           16655 non-null  category      
 2   Pricing, Delivery_Terms_Quote_Appr  16655 non-null  int64         
 3   Pricing, Delivery_Terms_Approved    16655 non-null  int64         
 4   Bureaucratic_Code_0_Approval        16655 non-null  int64         
 5   Bureaucratic_Code_0_Approved        16655 non-null  int64         
 6   Submitted_for_Approval              16655 non-null  int64         
 7   Bureaucratic_Code                   16655 non-null  category      
 8   Account_Created_Date                16655 non-null  datetime64[ns]
 9   Billing_Country                     16655 non-null  category      
 10  Account_Name      

In [13]:
sarasa = entrenamiento.dtypes.to_frame()
sarasa['1'] = test.dtypes
sarasa

,0,1
Region,category,category
Territory,category,category
"Pricing, Delivery_Terms_Quote_Appr",int64,int64
"Pricing, Delivery_Terms_Approved",int64,int64
Bureaucratic_Code_0_Approval,int64,int64
Bureaucratic_Code_0_Approved,int64,int64
Submitted_for_Approval,int64,int64
Bureaucratic_Code,category,category
Account_Created_Date,datetime64[ns],datetime64[ns]
Billing_Country,category,category


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2551 entries, 718 to 2550
Data columns (total 38 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Region                              2551 non-null   category      
 1   Territory                           2551 non-null   category      
 2   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64         
 3   Pricing, Delivery_Terms_Approved    2551 non-null   int64         
 4   Bureaucratic_Code_0_Approval        2551 non-null   int64         
 5   Bureaucratic_Code_0_Approved        2551 non-null   int64         
 6   Submitted_for_Approval              2551 non-null   int64         
 7   Bureaucratic_Code                   2551 non-null   category      
 8   Account_Created_Date                2551 non-null   datetime64[ns]
 9   Billing_Country                     2551 non-null   category      
 10  Account_Name          

In [15]:
test.dtypes

Region                                      category
Territory                                   category
Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Submitted_for_Approval                         int64
Bureaucratic_Code                           category
Account_Created_Date                  datetime64[ns]
Billing_Country                             category
Account_Name                                category
Opportunity_Name                            category
Opportunity_ID                                 int64
Account_Owner                               category
Opportunity_Owner                           category
Account_Type                                category
Opportunity_Type                            category
Delivery_Terms                              category
Opportunity_Created_Date              datetime

In [16]:
entrenamiento.Stage

16104    Closed Lost
16105    Closed Lost
16106    Closed Lost
16107    Closed Lost
16109     Closed Won
            ...     
16099     Closed Won
16100     Closed Won
16103     Closed Won
16101     Closed Won
16102     Closed Won
Name: Stage, Length: 16655, dtype: category
Categories (2, object): ['Closed Lost', 'Closed Won']

In [17]:
a = len(entrenamiento.columns)
b = len(test.columns)

sub_b = entrenamiento.columns.difference(test.columns)
sub_c = test.columns.difference(entrenamiento.columns)

print(a, b)
print(sub_b, sub_c)

39 38
Index(['Stage'], dtype='object') Index([], dtype='object')
